# 2. The Binding Polynomial

In the previous notebook we introduced the binary vector notation for site-specific states.  

Now we put this together with a non-redundant and hierarchical parameterization of the equilibrium constants to obtain a site-specific binding polynomial for a given set of states. 

In order to utilize the binding polynomial we need to specify the concentration of each state in terms of the concentration of the unliganded state (the reference state), the concentration of free ligand, and the thermodynamic parameters (equilibrium constants). The expression for a liganded state, represented by the binary vector, $\mathbf{B}$, is given by:

$$
[M_\mathbf{B}] = [M_\mathbf{0}]\prod_{
\substack{\mathbf{b} \\ 
\forall B_j=0,b_j=0 \\
\forall B_j=1,b_j\in\{0,1\}}}
f(\mathbf{b})
$$

$$ 
f(\mathbf{b}) = \begin{cases}
1 & \sum{b_j}=0 \\
K_\mathbf{b}[L_{i}] & \sum{b_j}=1, b_i=1 \\
\alpha_\mathbf{b} & \sum{b_j}>1
\end{cases}
$$

where $[M_\mathbf{0}]$ is the concentration of the unliganded state, $[L_i]$ is the concentration of free ligand for site $i$, and $K_\mathbf{b}$ and $\alpha_\mathbf{b}$ are the equilibrium constants.

This equation is used in coded below below to print the binding polynomial.

**Use the slider to vary the number of sites and click Print to display the binding polynomial.**

In [122]:
from IPython.display import display, Math
import itertools
import re

def f_b(b):
    '''
    The function 'f(b)' for determining the equilibrium constants
    based on an binary vector state, b.
    '''
    if(b.count('1')==0):
        return ''
    elif(b.count('1')==1):
        i = b.find('1')+1
        return 'K_{{{}}}[L_{{{}}}]'.format(b,i)
    elif(B.count('1')>1):
        return '\\alpha_{{{}}}'.format(b)

out=widgets.Output()
slider=widgets.IntSlider(value=3,min=1,max=5)
button=widgets.Button(description='Print')
vbox=widgets.VBox(children=[slider,button,out])
display(vbox)

def print_bp(b=None):
    
    n_sites=slider.value
    
    all_states = [''.join(seq) for seq in 
                  itertools.product('01', repeat=n_sites)]
    
    grouped_states = [[] for i in range(n_sites+1)] # initialize list
    
    for state in all_states:                        # count number
        n_ligands = state.count('1')                # of occupied sites
        grouped_states[n_ligands].append(state)
    
    [i.reverse() for i in grouped_states]           # swap order
    
    global states_flat
    states_flat = [item for sublist in grouped_states for item in sublist]
    bp = '[M_{{{}}}](1 + '.format(states_flat[0][0])
    for B in states_flat[1:]:
        # determine the number of occupied sites for the state
        # which will take on values of 0 or 1 in the product
        n_occ = B.count('1')
        
        # find the indices of the occupied sites
        indices = [m.start() for m in re.finditer('0',B)]
        
        # find all the sub-states for the given number of occupied sites
        b_sub = [''.join(seq) for seq in 
                      itertools.product('01', repeat=n_occ)]
        b_sub.reverse()
        
        # iterate through the sub-states and put these in the product
        for sub in b_sub:
            # add in the unoccupied sites
            for i,index in enumerate(indices):
                sub = sub[:index]+'0'+sub[index:]
            bp = bp + f_b(sub)
        
        # add each state
        bp = bp + ' + '
        
    # finishing touches on the binding polynomial
    bp = bp[:-3] + ')'
    # print the binding polynomial in nice format
    with out:
        clear_output(wait=True)
        display(Math(bp))

button.on_click(print_bp)
print_bp()

To illustrate the effect of the model parameters ($K$,$\alpha$) we now simulate the concentrations of each ligation state as a function of the free ligand concentration (not the total ligand concentration). To minimise computational time and keep things relatively simple we limit ourselves to homotropic ligand binding.

**Use the sliders below to vary the value of each parameter (log scale) then click the Plot button.**

Tips: Three sites provides a managable amount of parameters but retains complexity. Some of the lines overlap, make each $K$ slightly different to see each species.

In [123]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Layout,Label
from IPython.display import display,clear_output
import numpy as np
from matplotlib import pylab as plt
import matplotlib

%matplotlib inline
# To prevent automatic figure display when execution of the cell ends
%config InlineBackend.close_figures=False 

#states_flat = [item for sublist in grouped_states for item in sublist]
params = dict((k,1.) for k in states_flat[1:])

L = np.logspace(-6,6)

for B in states_flat[1:]:
    params[B] = widgets.FloatSlider(value=0.,min=-2,max=2,step=0.2)
    if(B.count('1')==1):
        params[B].description = r'\(K_{{{}}}\)'.format(B)
    else:
        params[B].description = r'\(\alpha_{{{}}}\)'.format(B)

def f_b_species(b):
    '''
    The function 'f(b)' for determining the equilibrium constants
    based on an binary vector state, b, modified to allow computation 
    of the species
    '''
    if(b.count('1')==0):
        return 1.
    elif(b.count('1')==1):
        return (10**params[b].value)*L
    elif(B.count('1')>1):
        return 10**(params[b].value)

matplotlib.rc('font', **{'size':22})

plt.ioff()
ax=plt.gca()

out=widgets.Output(layout=Layout(height='300px', width = '400px'))
button=widgets.Button(description='Plot',layout=Layout(width='400px'))
vbox1=widgets.VBox(children=[k for k in params.values()])
vbox2=widgets.VBox(children=[button,out])
hbox=widgets.HBox([vbox1,vbox2])
display(hbox)
    
def plot_species(b=None):
    concs = dict((k,1.) for k in states_flat)
    bp = 1.
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))    
    for B in states_flat[1:]:
            
        # determine the number of occupied sites for the state
        # which will take on values of 0 or 1 in the product
        n_occ = B.count('1')
            
        # find the indices of the occupied sites
        indices = [m.start() for m in re.finditer('0',B)]
            
        # find all the sub-states for the given number of occupied sites
        b_sub = [''.join(seq) for seq in 
                          itertools.product('01', repeat=n_occ)]
        b_sub.reverse()
            
        # iterate through the sub-states and put these in the product
        for sub in b_sub:
            # add in the unoccupied sites
            for i,index in enumerate(indices):
                sub = sub[:index]+'0'+sub[index:]
            concs[B] = concs[B]*f_b_species(sub)
        bp = bp + concs[B]
    
    for B in states_flat:
        ax.semilogx(L,concs[B]/bp, label=B)
    ax.set_xlabel('[Free Ligand]')
    ax.set_ylabel('Fraction in state')
    ax.legend()
    with out:
        clear_output(wait=True)
        display(ax.figure)

button.on_click(plot_species)
plot_species()